In [1]:
from helpers import *
import pandas as pd 
import numpy as np 
import re
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my_app3", timeout=5)
import time

In [2]:
#Realizar Web Scraping 
"""
Region: Recomendable escribir nombre de region igual al configurado  por defecto en la pagina de portal inmobiliario
        fijarse en url de busqueda, por ejemplo: al escribir santiago en buscador principal, la pagina redirige a los resultados con la         categoria "santiago-metropolitana"
pagina principal: https://www.portalinmobiliario.com/
ejemplo nombre de region url: https://www.portalinmobiliario.com/arriendo/departamento/santiago-metropolitana
ejemplo nombre de region url con rango de precio: https://www.portalinmobiliario.com/arriendo/departamento/santiago-metropolitana/_PriceRange_300000CLP-0CLP


Pages: Cantidad de paginas de resultados para la busqueda de ofertas,  no se puede superar el numero de paginas de resultados que existen en portalinmobiliario según la region especificada.

Scopes: pueden ser arriendo o venta.

type: puede ser departamento, casa, sitio, parcela, oficina, comercial, industrial, agricola, terreno-en-construccion, bodega, estacionamiento,  loteo, otros 
(considerar que webscraping esta optimizado para departamentos, por lo que puede que al utilizar otro tipo de inmueble algunas columnas como baños, habitaciones,etc podrian no coincidir, debido a que la información para cada tipo de inmueble es distinta)
"""
rango_precio="_PriceRange_300001CLP-370000CLP"
df = webscraping_deptos(region="santiago-metropolitana",pages=40,type='departamento',scope='arriendo', rango_precio=rango_precio)
print(df)

_PriceRange_300001CLP-370000CLP
Web Scraping Portal Inmobiliario
Buscando departamento para arriendo....


KeyboardInterrupt: 

In [ ]:
from datetime import datetime

now = datetime.now()
formatted_time = now.strftime("%Y-%m-%d %H:%M:%S")

df.to_csv('./data/'+'data-raw_'+formatted_time+'_'+rango_precio+'_'+'.csv',index=False)

: 

In [ ]:
def clean_data2(df):
    df['direccion'] = df['direccion'].str.replace("/",",")


    return df

: 

In [ ]:
df = clean_data2(df)

: 

In [ ]:
def clean_data(df):
    df = df[df.titulo.notnull()]
    df['titulo'] = df['titulo'].str.replace(r"\n|\t","")
    df['titulo'] = df['titulo'].str.replace(r"\n|\t","")
    df['descripcion'] = df['descripcion'].str.replace(r"\n|\t","")
    df['ambiente'] = df['ambiente'].str.replace(r"\n","").str.replace(r"\t\t","; ").str.replace(r"\t","")
    df['comodidades'] = df['comodidades'].str.replace(r"\n","").str.replace(r"\t\t","; ").str.replace(r"\t","")
    df['otras caracteristicas'] = df['otras caracteristicas'].str.replace(r"\n","").str.replace(r"\t\t","; ").str.replace(r"\t","")
    df['precio'] = df['precio'].str.replace(".","")
    df['gastos comunes'] = df['gastos comunes'].str.replace("CLP","")
    df['habitaciones'] = df['habitaciones'].str.replace(r"-\s+\d\s\w*","").str.replace('Dormitorio',"").str.strip()
    df['banos'] = df['banos'].str.replace(r"-\s+\d\s\w*","").str.replace(r'Baño|s',"").str.strip()
    df['direccion'] = df['direccion'].str.replace("/",",")
    df['m2_utiles'] = df['m2_utiles'].str.replace(r"-.*$|m.","")

    return df

: 

In [ ]:
#df = clean_data(df)

: 

In [ ]:
df.to_csv('./data/'+'clean-data'+formatted_time+rango_precio+'_'+'.csv',index=False)

: 

In [ ]:
import pandas as pd
#rango_precio="_PriceRange_0CLP-300000CLP"
csv_file_path = './data/'+'clean-data'+'2023-05-17 23:13:52'+'_'+rango_precio+'_'+'.csv'

df = pd.read_csv(csv_file_path)
print(len(df))

: 

In [ ]:
#Agregar coordenadas geograficas de direcciones a Data Frame con libreria geopy
targets_address= df['direccion'].tolist()

print(len(targets_address))

latitude = []
longitude = []
invalid_targets = []

for target in targets_address:
    target = target.split(",")[0] + ", Santiago, Chile"
    max_tries = 10
    retry_timeout = 5  # Timeout in seconds
    location = None  # Initialize location outside the loop
    for try_count in range(1, max_tries + 1):
        try:
            location = geolocator.geocode(target)
            break  # Break out of the loop if geocoding is successful
        except Exception as e:
            print("An error occurred:", e)
            print("error number: "+ str(try_count))
            pass

        if try_count < max_tries:
            sleep(retry_timeout)  # Wait for the specified timeout before retrying

    '''
    while True:
        try:
            location = geolocator.geocode(target)
            break  # Break out of the loop if geocoding is successful
        except:
            print("error in geolocator")
            continue  # Retry if an error occurs
    '''
    if location == None:
        invalid_targets.append(target)
        latitude.append(np.nan)
        longitude.append(np.nan)
    else:
        latitude.append(location.latitude)
        longitude.append(location.longitude)

#from datetime import datetime

#now = datetime.now()
#formatted_time = now.strftime("%Y-%m-%d %H:%M:%S")

with open("invalid_targets_"+formatted_time+".txt", "w") as file:
    for item in invalid_targets:
        file.write(str(item) + "\n")

: 

In [ ]:
#Almacenar resultados en Data Frame
print(len(latitude))
df['latitud'] = latitude
df['longitud'] = longitude

: 

In [ ]:
#Crear nuevo data frame con  direcciones validas y coordeandas geograficas no nulas 
geo_df = df[~df['latitud'].isnull()]
geo_df['ciudad'] = 'Santiago'

: 

In [ ]:
#formatted_time = 'new_way'
geo_df.to_csv('./data/'+'santiago_locations_'+formatted_time+'_'+rango_precio+'_'+'.csv',index=False)

: 